In [ ]:
#This was an example challenge to get familiar with making bootstrapping samples
#Bootstrapping samples is a random sample of the original data
#data points may be duplicated

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib notebook

In [ ]:
# Let's generate 1000 events drawn from a Gaussian distribution and calculate the mean and std dev

npts = 1000
mu = 10.0
sigma = 1.0

data = np.random.normal(mu,sigma,npts)

print "calculated mean   : %f" % (np.mean(data))
print "calculated std dev: %f" % (np.std(data))


In [ ]:
plt.figure(figsize=(4,4))
plt.hist(data,bins=50);

In [ ]:
# Now, generate a "fake" dataset of 1000 points randomly drawn from data. 
# Because you are pulling out 1000 points (the same number as in data), you will have some
# repeating data points. That's OK!

fake_data=np.random.choice(data,1000)

# When you make this fake dataset, calculate the mean

print np.mean(fake_data)
# YOUR CODE HERE

In [ ]:
# Now do this 1000 times, and keep track of all the different means from these mock datasets. 
# Histogram those means. 
plt.figure()
means=[]
stds=[]
data=np.array(data)
for x in range(1000):
    fake_data=np.random.choice(data,1000)
    stds.append(np.std(fake_data))
    means.append(np.mean(fake_data))
plt.hist(means, bins=50)
print 'average mean: ', np.mean(means)
print 'avergae standard deviation:', np.mean(stds)
# YOUR CODE HERE

In [ ]:
# What is the mean of those distributions?

# What are the values around the mean that enclose 68% of these mock data means? 95%?
'''
diff= abs(np.mean(stds)-stds)
diff.sort()
diff[int(1000*.68)]
print np.mean(stds)-diff[int(1000*.68)]
print np.mean(stds)+diff[int(1000*.68)]
'''
diff1= np.mean(stds)-stds
diff1.sort()
print diff1[(int(1000*.34)-1)]+np.mean(stds)
#print diff1
print diff1[(int(1000*.84)-1)] +np.mean(stds)
print 'mean of stds:',np.mean(stds)
print 'std of stds',np.std(stds)



In [ ]:


def bootstrapping(data):
    npts = len(data)
    indices = np.random.randint(0,npts,npts)

    bs_data= np.array([data[indices].copy()])
        
    return bs_data

# A product of two Gaussians
def signal_2D(npts,means,sigmas):
    pts = []
    for m,s in zip(means,sigmas):
        pts.append(np.random.normal(m,s,npts))
    pts = np.array(pts)
    return pts

In [ ]:
nMC_sig=1000
sigmeans=[5.0,7.0]
sigwidths=[0.1,0.1]
nsig_iteration=200

sig_mean=10.4
sig_width=.06
signal = np.random.normal(sig_mean,sig_width,nsig_iteration)
signal_points= signal_2D(nMC_sig,sigmeans,sigwidths)

print len(signal)
bootstrapping(np.array(signal))

In [1]:
import fitting_tools as ft
import time
import numpy as np
import matplotlib.pylab as plt
import random
%matplotlib notebook
import fitting_tools_main as ftm


niterations=1
sig_points=200
nbkg=1000
MC_signal=1000
MC_background=1000
dimension=1
sigmeans=10.4
sig_widths=.06
#sig_widths=[0.1,0.1]
#sigmeans = [5.0,7.0]

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 22 days


In [2]:
num_bootstrapping_samples=10


nneigh=3
start = time.time()
pulls,org_vals,fit_vals,fit_uncerts,iteration,file_name= ftm.calc_pull_main(niterations, sig_points, nbkg,MC_signal, MC_background,num_bootstrapping_samples,nneigh,sig_widths,sigmeans,dimension,tag='default')
print "time to calc %d iterations: %f s" % (niterations, time.time() - start)
print ''

writing out to file frac_values_default_sig200_bkg1000_MCsig1000_MCbkg1000_bs10_nn3_1dimension.dat
prob not bs
[ 1.16984912  1.69624255  1.92198911 ...,  0.32348589  0.15736326
  0.76515185]
[  3.82691968e+03   7.08497314e+03   5.72992334e+03 ...,   1.74989569e+00
   1.21900511e+00   5.39030495e+01]
[ 505.58148193  349.05990601  970.0055542  ...,  403.91989136  196.39932251
  942.96405029]
0.2

prob bs 
[ 0.58450955  2.51970957  3.05181074 ...,  0.52825085  0.1573551   0.367247  ]
prob bs 
[ 2.53140659  1.32374027  1.26956563 ...,  0.52826741  0.12763345
  0.25435227]
prob bs 
[ 0.90838782  1.21629175  3.05181074 ...,  0.52826741  0.09222415
  1.01300375]
prob bs 
[  1.32400256   3.98054113  14.65479785 ...,   4.43876678   0.09222415
   0.76515185]
prob bs 
[ 0.94129982  3.96475112  1.03716907 ...,  0.32348589  0.13491402
  1.01300375]
prob bs 
[ 2.20752832  1.49852234  3.90478799 ...,  0.32348589  0.37060176
  1.01300375]
prob bs 
[ 2.53140659  4.09225144  1.92198911 ...,  0.18468297 

In [ ]:
#Calculate the mean and std of frac pulls
def calc_mean_std_of_pulls(values):
    pulls=(values[0]-values[1])/values[2]
    return (np.mean(pulls),np.std(pulls))

print ""
print "bootstrapping samples"
infile_name='frac_values_default_sig200_bkg1000_MCsig1000_MCbkg1000_bs10_nn3_1dimension.dat'
values=np.loadtxt(infile_name, delimiter=' ', unpack=True,dtype=float)
print calc_mean_std_of_pulls(values)
print values[1]